In [ ]:
# setup
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import linalg

In [ ]:
# setup: has to be in a separate cell for some reason
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
# from before
def phase_plot(genfn, xlim, ylim, nx, ny, **kwargs):
    X, Y = np.meshgrid(np.linspace(xlim[0], xlim[1], nx),
                       np.linspace(ylim[0], ylim[1], ny))
    X.shape = Y.shape = (np.prod(X.shape),)
    U, V = genfn(X, Y, **kwargs)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.quiver(X, Y, U-X, V-Y) #, angles='xy', scale_units='xy', scale=1)
    return fig

# Linear algebra, age structure, and eigenpopulations

Next we'll be learning some *linear algebra*, which gives us
the tools we need to generalize what we've done so far to higher dimensions.
We'll follow Otto & Day in doing this in the context of 
*age* and *stage structured populations*.

We're also going to focus on the *short term* dynamics of these populations:
for reasons that will be clear later, we won't be concerned about long-term stability.


# Linear algebra

A quick intro on the board to:

1. Vector addition (`+`): vectors as 
2. Matrix-vector multiplication (`@`)
3. Matrix-matrix multiplication (`@`)
4. Matrices as systems of linear equations (`scipy.linalg.solve( )`)

Problems: let
$$
  M = \begin{bmatrix} 1 & 2 & 3 \\ 0 & 4 & 5 \\ 0 & 0 & 6 \end{bmatrix}
$$
and
$$\begin{aligned}
 x = \begin{bmatrix} 3 \\ -6 \\ 9 \end{bmatrix}
 & \qquad &
 y = \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix} .
\end{aligned}$$


1. Find $x + y$.
2. Find $M x$.
3. Show that $M(x+y) = Mx + My$.
4. Show that $M(Mx) = (M^2)x$.
5. Find the vector $u$ so that $Mu = x$.


# Life stages (aka class-structured populations)

Let's go back to thinking about a single species,
but keeping track not just of how many individuals there are,
but also what *life stage* or *class* they are all in.
We *could* keep track of this just as a list of life stages -
if there are $N$ individuals, this would be a list of $N$ numbers.
But it's tidier to keep track of the *census*:
for each possible class, how many individual are there of that age?
We'll write that as follows:
$$
  N_k(t) = \text{(number of class-$k$ individuals alive at time $t$)} .
$$


## Example: juveniles/adults

Let's start with the simplest example.
- Every individual is either *juvenile* (class $J$) or *adult* (class $A$).
- Juveniles *mature* into adults with probability $p_j$, or else die.
- Adults give birth to on average $b$ offspring per time step, and
- survive to the next time step with probability $p_a$.

The discrete-time (deterministic) equation is
$$ \begin{aligned}
  N_J(t+1) &= 0 N_J(t) +  b N_A(t) \\
  N_A(t+1) &= p_j N_J(t) + p_a N_A(t) .
\end{aligned} $$
Using the matrix
$$\begin{aligned}
M =
\begin{array}{c c} & \begin{array}{@{} c c @{}} J\hphantom{0} & \hphantom{0}A \end{array} \\
    \begin{array}{c} J \\ A \end{array}\hspace{-1em} &
    \left(
        \begin{array}{@{} c  @{}}
        0 & b \\
        p_j & p_a
        \end{array}
    \right) \\ \mbox{} \end{array} \\[-12pt] ,
\end{aligned}$$
and $N = (N_J, N_A)$,
we can write this same system of equations concisely as
$$
N(t+1) = M N(t) .
$$

**What's this system do?**

In [ ]:
# The parameters
b = 10
pj = 0.1
pa = 0.2
M = np.array([[0, b], [pj, pa]])
print(M)

In [ ]:
M @ eM[1][:,0] / eM[0][0]

In [ ]:
# suppose we start with one juvenile and 9 adults?
print(M @ np.array([1, 9]))

In [ ]:
# iterate these equations
N0 = [1, 9]
ntimes = 20
N = np.empty((ntimes, 2), dtype='float')
N[0,:] = N0
for j in range(1, ntimes):
    N[j, :] = M @ N[j-1, :]
    
fig, ax = plt.subplots()
ax.plot(N)
ax.set_xlabel("time")
ax.set_ylabel("numbers")
ax.legend(['juveniles', 'adults']);

# Eigendirections

*Eigenvectors* of a matrix are directions *unchanged* by its transformation.
In symbols, this is
$$
  Mv = \lambda v
$$
where $\lambda$ is a *number*, called the *eigenvalue*.

Exercises:

1. Check that $(1, 0)$ and $(0, 1)$ are eigenvectors of
    $$
    M = \begin{bmatrix} 2 & 0 \\ 0 & 3 \end{bmatrix} .
    $$
    What are the eigenvalues?

2. Use `scipy.linalg.eig( )` to find the eigenvalues and vectors of 
    $$
      M = \begin{bmatrix} 1 & 2 & 3 \\ 0 & 4 & 5 \\ 0 & 0 & 6 \end{bmatrix}
    $$
    Double-check the equation holds.

3. The formula corresponding to "change into the coordinates of the eigenvectors,
    multiply by the eigenvalues, and change coordinates back" is
    $$
      M = U \text{diag}(\lambda) U^{-1} ,
    $$
    where $U^{-1}$ is the *inverse matrix* (`scipy.linalg.solve(U, np.eye(U.shape[0]))`)
    and $\text{diag}(\lambda)$ is the matrix with the values of $\lambda$ on the diagonal
    and zeros elsewhere.
    Check this holds.


In [ ]:
M = np.array([[2, 1], [1/2, 1]])
lam, U = linalg.eig(M)

def lin_transform(X, Y, M):
    out = M @ np.row_stack([X, Y])
    return out[0, :], out[1, :]

plt.rcParams['figure.figsize'] = [8, 8]
fig = phase_plot(lin_transform, [-1, 1], [-1, 1], 20, 20, M=M)
ax = fig.gca()
plt.axis("equal")
for x, y in U.T:
    ax.plot([-x, x], [-y, y])

fig.legend(['evector1', 'evector2']);


## Example: right whales

For instance, the classes for females in a right whale population model were:

- class `C`: calves
- class `I`: immature
- class `M`: mature
- class `R`: reproductive

Individuals transition from calf to immature in one year;
can stay immature for more than one year;
then transition between mature and reproductive as adults.
Each class has a separate death rate,
and only reproductive individuals produce new offspring.
The parameters of the model are concisely encoded
in the following matrix:
$$
S =
\begin{array}{c c} & \begin{array}{@{} c c c c @{}} C\hphantom{0} & I\hphantom{0} & \hphantom{0}M & \hphantom{0}R \end{array} \\
    \begin{array}{c} C \\ I \\ M \\ R \end{array}\hspace{-1em} &
    \left(
        \begin{array}{@{} c c c c @{}}
        0 & 0 & 0 & b \\
        s_{IC} & s_{II} & 0 & 0 \\
        0 & s_{MI} & s_{MM} & s_{MR} \\
        0 & s_{RI} & s_{RM} & s_{RR} 
        \end{array}
    \right) \\ \mbox{} \end{array} \\[-12pt] 
$$
There are nine parameters.
All the $s_{XY}$ parameters have the same interpretation: they give the probability
that a given individual *currently in class $Y$* will be *in class $X$* next time step.
For instance, a calf will either
- become immature (with probability $s_{IC}$, or
- die (otherwise),

while a mature individual will either
- stay immature (probability $s_{II}$,
- become mature (probability $s_{MI}$, or
- become reproductive ($s_{RI}$).

The remaining parameter, $b$, is the mean number of *births* per reproductive individual per year.